In [ ]:
def csviter(obj, delimiter=None, headers=None, encoding=None, quotechar=None):
    """ Returns an iterator of dictionaries from the given csv object

    obj can be:
    - a Response object
    - a unicode string
    - a string encoded as utf-8

    delimiter is the character used to separate fields on the given obj.

    headers is an iterable that when provided offers the keys
    for the returned dictionaries, if not the first row is used.

    quotechar is the character used to enclosure fields on the given obj.
    """

    encoding = obj.encoding if isinstance(obj, TextResponse) else encoding or 'utf-8'
    def _getrow(csv_r):
        return [to_unicode(field, encoding) for field in next(csv_r)]

    # Python 3 csv reader input object needs to return strings
    if six.PY3:
    # lines is a decoded version of obj     
        lines = StringIO(_body_or_str(obj, unicode=True))
    else:
        lines = BytesIO(_body_or_str(obj, unicode=False))

    kwargs = {}
    if delimiter: kwargs["delimiter"] = delimiter
    if quotechar: kwargs["quotechar"] = quotechar
    # csv.reader(lines) return a reader object which will iterate over lines of lines,  **kwargs supplies the law of separation
    # Each row read from the csv file is returned as a list of strings
    ''' >>>x = open ('text.txt','w')
        >>>x.write('a b c d e f\n')
        >>>x.write('1 2 3 4 5 6')
        >>>x.close()
        >>>import csv
        >>>rules = {'delimiter': ' ' }
        >>>with open('text.txt', newline='') as csvfile :
        >>>spamreader = csv.reader(csvfile, **rules)
        >>>print (list(spamreader))
        [['a', 'b', 'c', 'd', 'e', 'f'], ['1', '2', '3', '4', '5', '6']]
    '''
    csv_r = csv.reader(lines, **kwargs)


    if not headers:
        headers = _getrow(csv_r)

    while True:
        row = _getrow(csv_r) 
        if len(row) != len(headers):
            logger.warning("ignoring row %(csvlnum)d (length: %(csvrow)d, "
                           "should be: %(csvheader)d)",
                           {'csvlnum': csv_r.line_num, 'csvrow': len(row),
                            'csvheader': len(headers)})
            continue
        else:
            yield dict(zip(headers, row))



# ~/environments/web_crawling_env/lib/python2.7/site-packages/scrapy/utils